## QUESTION 1

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
wiki_site = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_site, 'lxml')
print (soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: deprecated parameters","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","Feb

# Tabular data belongs to class="wikitable sortable", extracting just the table

In [6]:
Post_table = soup.find('table', {'class':'wikitable sortable'})
Post_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [7]:
print(Post_table.tr.text)


Postcode
Borough
Neighbourhood



# Getting all values in  tr and separating all td with commas(",")
    

In [9]:
Table1=""
for tr in Post_table.find_all('tr'):
    rows=""
    for tds in tr.find_all('td'):
        rows=rows+","+tds.text
    Table1=Table1+rows[1:]
print(Table1)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M

# Writting Data into .CSV file


In [11]:
file=open("Toronto.csv", "wb")
file.write(bytes(Table1,encoding="ascii",errors="ignore"))


8738

In [13]:
df=pd.read_csv("Toronto.csv", header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]

In [15]:
df.head(5)

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Removing rows that do not have Borough values i.e Not assigned

In [21]:
rowNames = df[df["Borough"]=="Not assigned"].index
df.drop(rowNames, inplace=True)
df.head(5)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [24]:
df.loc[df["Neighbourhood"] == "Not assigned", "Neighbourhood"] = df["Borough"]
df.head(5)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


# Combine similar Postal code with different Neighbourhoods into one row separated by ","

In [26]:
Tab = df.groupby(["Postalcode", "Borough"], sort=False).agg( ', '.join)
df1 = Tab.reset_index()
df1.head(10)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [27]:
df1.shape

(103, 3)

## QUESTION 2

Using the csv file to create a data frame of Long and Lat values

In [31]:
!wget -q -O "Toronto_coordinates.csv" https://cocl.us/Geospatial_data

In [33]:
df_cord = pd.read_csv("Toronto_coordinates.csv")
df_cord.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
df_cord.columns=["Postalcode", "Latitude", "Longitude"]
df_cord.head(5)

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
Toronto_df = pd.merge(df1, df_cord[["Postalcode", "Latitude", "Longitude"]], on=["Postalcode"])
Toronto_df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## QUESTION 3

Exploring and clustering the neighborhoods in Toronto

In [43]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [44]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [45]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [47]:
CLIENT_ID = 'ORSZBNQF0PXM0IUHWX4CXWB411QWBC3R50GYDK3BS3KCUIP4' 
CLIENT_SECRET = 'HXVUKUHMX5MPE4SD3AHPWCCN4BH1KKNTDNDQDK330NCYFG0P' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ORSZBNQF0PXM0IUHWX4CXWB411QWBC3R50GYDK3BS3KCUIP4
CLIENT_SECRET:HXVUKUHMX5MPE4SD3AHPWCCN4BH1KKNTDNDQDK330NCYFG0P


In [48]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [49]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=ORSZBNQF0PXM0IUHWX4CXWB411QWBC3R50GYDK3BS3KCUIP4&client_secret=HXVUKUHMX5MPE4SD3AHPWCCN4BH1KKNTDNDQDK330NCYFG0P&v=20180604&ll=43.653963, -79.387207&radius=500&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighbourhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

Size of the Dataframe

In [51]:
toronto_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,Parkwoods,43.753259,-79.329656,Japango,43.655268,-79.385165,Sushi Restaurant
2,Parkwoods,43.753259,-79.329656,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
3,Parkwoods,43.753259,-79.329656,Cafe Plenty,43.654571,-79.389450,Café
4,Parkwoods,43.753259,-79.329656,Poke Guys,43.654895,-79.385052,Poke Place
5,Parkwoods,43.753259,-79.329656,Tsujiri,43.655374,-79.385354,Tea Room
6,Parkwoods,43.753259,-79.329656,Textile Museum of Canada,43.654396,-79.386500,Art Museum
7,Parkwoods,43.753259,-79.329656,Rolltation,43.654918,-79.387424,Japanese Restaurant
8,Parkwoods,43.753259,-79.329656,Manpuku まんぷく,43.653612,-79.390613,Japanese Restaurant
9,Parkwoods,43.753259,-79.329656,Karine's,43.653699,-79.390743,Breakfast Spot


In [54]:
toronto_venues.shape

(8137, 7)

In [57]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",79,79,79,79,79,79
Agincourt,79,79,79,79,79,79
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",79,79,79,79,79,79
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",79,79,79,79,79,79
"Alderwood, Long Branch",79,79,79,79,79,79
"Bathurst Manor, Downsview North, Wilson Heights",79,79,79,79,79,79
Bayview Village,79,79,79,79,79,79
"Bedford Park, Lawrence Manor East",79,79,79,79,79,79
Berczy Park,79,79,79,79,79,79


## Analysing each Neighbourhood

In [59]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Toy / Game Store,University,Vegetarian / Vegan Restaurant,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Parkwoods
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods


In [60]:
toronto_onehot.shape

(8137, 58)


grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category

In [61]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Breakfast Spot,Bubble Tea Shop,...,Sandwich Place,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
1,Agincourt,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
4,"Alderwood, Long Branch",0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
5,"Bathurst Manor, Downsview North, Wilson Heights",0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
6,Bayview Village,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
7,"Bedford Park, Lawrence Manor East",0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
8,Berczy Park,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316
9,"Birch Cliff, Cliffside West",0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316,0.037975,0.025316,...,0.025316,0.012658,0.012658,0.012658,0.037975,0.012658,0.012658,0.012658,0.012658,0.025316


In [62]:
toronto_grouped.shape

(103, 58)

#### Let's print each neighbourhood along with the top 5 most common venues

In [66]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  Japanese Restaurant  0.04
3          Art Gallery  0.04
4     Sushi Restaurant  0.04


----Agincourt----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  Japanese Restaurant  0.04
3          Art Gallery  0.04
4     Sushi Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  Japanese Restaurant  0.04
3          Art Gallery  0.04
4     Sushi Restaurant  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  Japanese Restaurant  0.04
3          Art Gallery  0.04
4     Sushi Restaurant  0.04


----Alderwood, Long Branch----
                 venue  freq
0 

#### Let's put that into a pandas dataframe

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [73]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
1,Agincourt,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
4,"Alderwood, Long Branch",Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant


## Cluster Neighborhoods

In [74]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [78]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'cluster labels', kmeans.labels_)

toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,cluster labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant


#### Visualizing the resulting cluster

In [85]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [86]:
toronto_merged.loc[toronto_merged['cluster labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
1,North York,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
2,Downtown Toronto,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
3,North York,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
4,Queen's Park,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
5,Etobicoke,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
6,Scarborough,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
7,North York,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
8,East York,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant
9,Downtown Toronto,0,0,Coffee Shop,Café,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant,Hotel,Chinese Restaurant,Bubble Tea Shop,Vegetarian / Vegan Restaurant


#### Cluster 2

In [92]:
toronto_merged.loc[toronto_merged['cluster labels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 3

In [88]:
toronto_merged.loc[toronto_merged['cluster labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 4

In [90]:
toronto_merged.loc[toronto_merged['cluster labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 5

In [91]:
toronto_merged.loc[toronto_merged['cluster labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
